<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Cloud Storage API (python)

In this notebook, we will show you how to use the [__Cloud Sorage API__](https://googleapis.dev/python/storage/latest/client.html) for python.

In particular you will learn how to:
  - create a bucket
  - upload an object
  - manage permissions to grant public access to a specific object or the entire bucket
  - download an object
  - delete an object
  - delete a bucket

### Install the libraries

In [26]:
pip install -q google-cloud

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install -q google-cloud-storage

Note: you may need to restart the kernel to use updated packages.


### Setup the environment

In [28]:
# Import the python libraries

import os
from google.cloud import storage

In [29]:
# Import the .json credentials and set the environment variable (TODO:student)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./credentials.json"

In [30]:
# Get the client reference

client = storage.Client()

## Create a bucket

__Note__: in order to avoid conflicts with your colleagues, please name the bucket_id as "\<surname\>-gcp-quantia-python"

In [31]:
# Set the bucket paramenters (TODO:student) 

project_id = "preparazione-lezione-gcp"
bucket_id = "gcp-quantia-python"
bucket_location = "EU"  # Multiregion (see https://cloud.google.com/storage/docs/locations) 
storage_class = "STANDARD" # STANDARD / NEARLINE / COLDLINE / ARCHIVE


In [32]:
# Call the API for creating the bucket

bucket = storage.Bucket(client, name=bucket_id)

bucket.project = project_id
bucket.location = bucket_location
bucket.storage_class = storage_class

bucket = client.create_bucket(bucket) # API request

print("Created bucket: {}".format(bucket.name))

<ipython-input-32-708116399844>:6: DeprecationWarning: Assignment to 'Bucket.location' is deprecated, as it is only valid before the bucket is created. Instead, pass the location to `Bucket.create`.
  bucket.location = bucket_location


Created bucket: gcp-quantia-python


In [33]:
# List all the buckets to check that has been created

buckets = client.list_buckets()

for bucket in buckets:
        print(bucket.name)

gcp-quantia-python
iot-heartrate-temp-quantia


## Upload an object

In [34]:
# Set the file path (TODO:student)

file_path = "./test.jpg"
destination_folders = "folder1/folder2/"
blob_name = "test-image.png"

In [35]:
# Upload the object

bucket = client.bucket(bucket_id)
blob = bucket.blob(blob_name)

blob.upload_from_filename(file_path)

print("File {} uploaded to {}.".format(file_path, blob_name) )

File ./test.jpg uploaded to test-image.png.


In [36]:
# Upload the same object, but inside some folders

bucket = client.bucket(bucket_id)
blob = bucket.blob(destination_folders + blob_name)

blob.upload_from_filename(file_path)

print("File {} uploaded to {}.".format(file_path, blob_name) )

File ./test.jpg uploaded to test-image.png.


In [37]:
# List all the files of the bucket to check the upload

blobs = client.list_blobs(bucket_id)

for blob in blobs:
    print(blob.name)

folder1/folder2/test-image.png
test-image.png


## Make a specific object publicly accessible

In [38]:
# Enable public pemission of the specific object

bucket = client.bucket(bucket_id)
blob = bucket.blob(blob_name)

blob.make_public()

print("Blob {} is publicly accessible at {}".format( blob.name, blob.public_url) )

Blob test-image.png is publicly accessible at https://storage.googleapis.com/gcp-quantia-python/test-image.png


In [39]:
# Remove public permission of the specific object

bucket = client.bucket(bucket_id)
blob = bucket.blob(blob_name)

blob.make_private()

print("Blob {} is no more publicly accessible".format( blob.name) )


Blob test-image.png is no more publicly accessible


## Make the entire bucket publicly accessible

In [40]:
# Set the permissions, get the bucket

bucket = client.bucket(bucket_id)

role = "roles/storage.objectViewer"
member = "allUsers"

In [41]:
# Make the bucket public

policy = bucket.get_iam_policy(requested_policy_version=3)
policy.bindings.append({"role": role, "members": {member}})

bucket.set_iam_policy(policy)

print("Bucket {} is now publicly readable".format(bucket.name))

Bucket gcp-quantia-python is now publicly readable


For testing the feature, you can try to access the object stored in the folder at https://storage.googleapis.com/BUCKET-ID/folder1/folder2/test-image.png   (note: change BUCKET-ID according to the name you have provided)

In [42]:
# Show the ACL list (check if "allUsers" has reading permission)

policy = bucket.get_iam_policy(requested_policy_version=3)

for binding in policy.bindings:
    print("Role: {}, Members: {}".format(binding["role"], binding["members"]))

Role: roles/storage.legacyBucketOwner, Members: {'projectOwner:preparazione-lezione-gcp', 'projectEditor:preparazione-lezione-gcp'}
Role: roles/storage.legacyBucketReader, Members: {'projectViewer:preparazione-lezione-gcp'}
Role: roles/storage.objectViewer, Members: {'allUsers'}


In [43]:
# Remove public permission

policy = bucket.get_iam_policy(requested_policy_version=3)
policy.bindings.remove({"role": role, "members": {member}})

bucket.set_iam_policy(policy)

print("Bucket {} is no more publicly readable".format(bucket.name))

Bucket gcp-quantia-python is no more publicly readable


In [44]:
# Show the ACL list (check if permissions of "allUsers" have been removed)

policy = bucket.get_iam_policy(requested_policy_version=3)

for binding in policy.bindings:
    print("Role: {}, Members: {}".format(binding["role"], binding["members"]))

Role: roles/storage.legacyBucketOwner, Members: {'projectOwner:preparazione-lezione-gcp', 'projectEditor:preparazione-lezione-gcp'}
Role: roles/storage.legacyBucketReader, Members: {'projectViewer:preparazione-lezione-gcp'}


## Download an object

In [45]:
# Set the file path (TODO:student)

destination_file_name = "./download.jpg"
source_blob_name = blob_name

In [46]:
bucket = client.bucket(bucket_id)
blob = bucket.blob(source_blob_name)
blob.download_to_filename(destination_file_name)

print("Downloaded public blob {} from bucket {} to {}.".format(source_blob_name, bucket.name, destination_file_name))

Downloaded public blob test-image.png from bucket gcp-quantia-python to ./download.jpg.


## Delete an object

In [47]:
# Set the file path (TODO:student)
blob_to_delete = "folder1/folder2/test-image.png"

In [48]:
# Delete the file located in blob_name

bucket = client.bucket(bucket_id)
blob = bucket.blob(blob_to_delete)
blob.delete()

print("Blob {} deleted.".format(blob_to_delete))

Blob folder1/folder2/test-image.png deleted.


## Delete a bucket

In [49]:
# Deletes the entire bucket

bucket = client.get_bucket(bucket_id)
bucket.delete(force=True) # Force deletion, even if bucket is non-empty

print("Bucket {} deleted".format(bucket.name))

Bucket gcp-quantia-python deleted


##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2021 Quantia Consulting, srl. All rights reserved.